In [ ]:
import json
import os
import shutil
from pathlib import Path
import sys
import time
import pyperclip

In [ ]:
def backup_large_media():
    user_profile = os.environ.get('USERPROFILE')
    
    tasks = {
        os.path.join(user_profile, "Pictures"): r"H:\windows-backup\media-backup\Pictures",
        os.path.join(user_profile, "Videos"): r"H:\windows-backup\media-backup\Videos",
        os.path.join(user_profile, r"Apple\MobileSync\Backup"): r"H:\windows-backup\media-backup\Apple"
    }

    ignore_files = ["desktop.ini"]
    ignore_folders = ["Camera Roll", "Saved Pictures", "Captures"]

    for src_root, dest_root in tasks.items():
        if not os.path.exists(src_root):
            print(f"Skipping: {src_root} (Not found)")
            continue

        print(f"Scanning {os.path.basename(src_root)}...")
        
        files_to_copy = []
        total_bytes_to_copy = 0
        
        # 1. SCANNING PHASE
        for root, dirs, files in os.walk(src_root):
            dirs[:] = [d for d in dirs if d not in ignore_folders]
            for file in files:
                if file in ignore_files or file.startswith(".lock_"):
                    continue
                
                src_file = os.path.join(root, file)
                rel_path = os.path.relpath(src_file, src_root)
                dest_file = os.path.join(dest_root, rel_path)
                
                # REPLACEMENT LOGIC:
                # 1. Copy if it doesn't exist
                # 2. OR Copy if size is different
                # 3. OR Copy if the source is newer than the backup
                should_copy = False
                if not os.path.exists(dest_file):
                    should_copy = True
                else:
                    src_stat = os.stat(src_file)
                    dest_stat = os.stat(dest_file)
                    if src_stat.st_size != dest_stat.st_size:
                        should_copy = True
                    elif src_stat.st_mtime > dest_stat.st_mtime + 1: # +1s buffer for filesystem jitter
                        should_copy = True

                if should_copy:
                    try:
                        file_size = os.path.getsize(src_file)
                        files_to_copy.append((src_file, dest_file, file_size))
                        total_bytes_to_copy += file_size
                    except OSError:
                        continue 

        # 2. COPYING PHASE
        num_files = len(files_to_copy)
        if num_files > 0:
            total_gb = total_bytes_to_copy / (1024**3)
            print(f"Transferring {num_files} files ({total_gb:.2f} GB)...")
            
            bytes_copied_so_far = 0
            start_time = time.time()
            
            for index, (src, dest, file_size) in enumerate(files_to_copy, 1):
                try:
                    os.makedirs(os.path.dirname(dest), exist_ok=True)
                    shutil.copy2(src, dest) # copy2 keeps timestamps, essential for our mtime check next run
                    
                    bytes_copied_so_far += file_size
                    percent = (bytes_copied_so_far / total_bytes_to_copy) * 100
                    current_gb = bytes_copied_so_far / (1024**3)
                    filename = os.path.basename(src)
                    
                    sys.stdout.write(
                        f"\rProgress: {percent:.2f}% | {current_gb:.2f}/{total_gb:.2f} GB ({index}/{num_files}) | File: {filename[:60]}"
                    )
                    sys.stdout.flush()
                    
                except Exception as e:
                    print(f"\n[Error] Could not copy {os.path.basename(src)}: {e}")
            
            end_time = time.time()
            duration = end_time - start_time
            print(f"\nDone! Backup took {duration/60:.1f} minutes.")
        else:
            print(f"Everything in {os.path.basename(src_root)} is already up to date.")

backup_large_media()

In [ ]:
def generate_config_paths():
    roaming = os.environ.get('APPDATA')
    local = os.environ.get('LOCALAPPDATA')
    
    folder_map = {}
    search_dirs = [("%appdata%", roaming), ("%localappdata%", local)]
    
    # 1. Gather and group folders
    for path_label, root_path in search_dirs:
        if root_path and os.path.exists(root_path):
            for item in os.listdir(root_path):
                full_path = os.path.join(root_path, item)
                if os.path.isdir(full_path):
                    if item not in folder_map:
                        folder_map[item] = []
                    # Using double backslashes for JSON compatibility
                    formatted_path = f"{path_label}\\\\{item}"
                    folder_map[item].append(formatted_path)

    # 2. Case-insensitive sort
    sorted_keys = sorted(folder_map.keys(), key=str.lower)

    # 3. Build the output string
    output_lines = []
    for i, folder_name in enumerate(sorted_keys):
        paths = folder_map[folder_name]
        
        output_lines.append(f'    "{folder_name}":')
        output_lines.append(f'    [')
        
        for j, p in enumerate(paths):
            path_comma = "," if j < len(paths) - 1 else ""
            output_lines.append(f'        "{p}"{path_comma}')
        
        # Add comma to the closing bracket unless it's the very last folder
        block_comma = "," if i < len(sorted_keys) - 1 else ""
        output_lines.append(f'    ]{block_comma}')

    final_output = "\n".join(output_lines)

    # 4. Print and Copy to Clipboard
    print(final_output)
    pyperclip.copy(final_output)
    print("\n" + "="*30)
    print("SUCCESS: Config copied to clipboard!")
    print("="*30)

# generate_config_paths()

In [ ]:
# 1. Load the settings
settings_file = 'windows-reset-settings.json'
with open(settings_file, 'r') as f:
    settings = json.load(f)

backup_root = Path(settings['backup-dir'])
apps = settings['applications']

print(f"Starting backup to: {backup_root}")

In [ ]:
# 2. Handle the 'regs' folder
source_regs = Path('reg-mods')
target_regs = backup_root / 'reg-mods'

if source_regs.exists():
    if target_regs.exists():
        print("Cleaning existing 'reg-mods' folder in backup directory...")
        shutil.rmtree(target_regs)
    
    shutil.copytree(source_regs, target_regs)
    print("Successfully backed up 'reg-mods' folder.")
else:
    print("Warning: Local 'reg-mods' folder not found. Skipping.")

In [ ]:
#3. install-applications.txt
def backup_install_list(backup_root):
    source_file = Path('install-applications.txt')
    destination_path = Path(backup_root) / 'install-applications.txt'
    
    if source_file.exists():
        try:
            shutil.copy2(source_file, destination_path)
            print(f"  [●] Finished:  install-applications.txt (Overwritten)")
        except Exception as e:
            print(f"  [!] Error backing up install-applications.txt: {e}")
    else:
        print(f"  [!] Skip: install-applications.txt not found in current folder.")

backup_install_list(backup_root)

In [ ]:
def force_copy_with_tracking(src, dst):
    """
    Recursively copies files. 
    Returns (success_count, fail_count)
    """
    s_count = 0
    f_count = 0
    
    src = Path(src)
    dst = Path(dst)

    if src.is_dir():
        try:
            dst.mkdir(parents=True, exist_ok=True)
        except Exception as e:
            print(f"    [!] Critical: Could not create directory {dst.name}: {e}")
            return 0, 1
            
        for item in os.listdir(src):
            s, f = force_copy_with_tracking(src / item, dst / item)
            s_count += s
            f_count += f
    else:
        try:
            shutil.copy2(src, dst)
            s_count += 1
        except Exception as e:
            print(f"    [!] Failed to copy file: {src.name} (Reason: {e})")
            f_count += 1
            
    return s_count, f_count

PATH_MAP = {
    "%appdata%": "appdata",
    "%localappdata%": "localappdata",
    "%userprofile%\\documents": "documents",
    "%programdata%": "programdata",
    "%userprofile%": "userprofile"
}

In [ ]:
# 4. Process Applications
for app_name, paths in apps.items():
    START_LABEL = "[+] Backing up:"
    FINISH_LABEL = "[»] Finished:  "
    
    print(f"\nProcessing application: {app_name}")
    
    # Create the app container in 'applications' (empty placeholder for notes/screenshots)
    (backup_root / 'info' / app_name).mkdir(parents=True, exist_ok=True)
    
    for raw_path in paths:
        # Determine the mirrored destination path
        mirror_subfolder = "misc"
        relative_path = ""
        clean_raw_path = raw_path.lower()
        
        for env_var, folder_name in PATH_MAP.items():
            if clean_raw_path.startswith(env_var.lower()):
                mirror_subfolder = folder_name
                # Extract the path after the environment variable
                relative_path = raw_path[len(env_var):].strip("\\")
                break
        
        # If no env var match, we just use a sanitized version of the whole path
        if not relative_path:
            relative_path = raw_path.replace('%', '').replace('\\', '_').replace(':', '').replace('.', '_').strip('_')

        expanded_path_str = os.path.expandvars(raw_path)
        source_path = Path(expanded_path_str)
        
        if not source_path.exists():
            print(f"  [!] Path not found: {expanded_path_str}")
            continue

        # The mirror destination: backups / appdata / discord
        destination_dir = backup_root / 'backups' / mirror_subfolder / relative_path
        
        # Atomic Update Logic
        temp_dir = None
        if destination_dir.exists():
            temp_dir = destination_dir.with_name(f"{destination_dir.name}_old_tmp")
            if temp_dir.exists(): shutil.rmtree(temp_dir)
            destination_dir.rename(temp_dir)
            
        destination_dir.mkdir(parents=True, exist_ok=True)
        
        print(f"  {START_LABEL} {mirror_subfolder}\\{relative_path}")
        
        try:
            # We copy the CONTENTS of source_path into destination_dir
            if source_path.is_dir():
                s_total, f_total = 0, 0
                for item in os.listdir(source_path):
                    s, f = force_copy_with_tracking(source_path / item, destination_dir / item)
                    s_total += s
                    f_total += f
                successes, failures = s_total, f_total
            else:
                # If the source is just a single file
                shutil.copy2(source_path, destination_dir / source_path.name)
                successes, failures = 1, 0

            print(f"  {FINISH_LABEL} {mirror_subfolder}\\{relative_path}; {successes} files copied, {failures} skipped")
            
            if temp_dir and temp_dir.exists():
                shutil.rmtree(temp_dir)
                
        except Exception as e:
            print(f"  [!] CRITICAL: {e}")
            if temp_dir and temp_dir.exists():
                print(f"  [!] Rollback: Restoring previous backup")
                if destination_dir.exists(): shutil.rmtree(destination_dir)
                temp_dir.rename(destination_dir)